In [1]:
!pip install simpletransformers # for transformers wrapper for DL
!pip install ktrain # for ktrain wrapper for DL
!pip install nlpaug # for data augmantation
!pip install tqdm

In [2]:
import numpy as np
import pandas as pd

import json, sys, regex, os
import torch
import torchvision
import random
random.seed(2022)

from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import requests
import re 

import os
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from tqdm import tqdm


import nltk
from nltk.corpus import stopwords
from collections import Counter


import ktrain
from ktrain import text

from sklearn.utils import shuffle
from transformers import TFRobertaModel, RobertaConfig, RobertaTokenizerFast

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

%config InlineBackend.figure_format = 'retina'

**I just need Happines, Sad, Angry and Natural classies.**

## Take emotions from 1st data.
[Emotion Detection from Text Data](https://www.kaggle.com/pashupatigupta/emotion-detection-from-text)


In [3]:
tweet_emotions_1 = pd.read_csv('../input/emotion-detection-from-text/tweet_emotions.csv')
tweet_emotions_1.head(5)

In [4]:
tweet_emotions_1.sentiment.value_counts()

In [5]:
happy_tweets_1   = tweet_emotions_1['content'][tweet_emotions_1['sentiment'] == 'happiness'].values.tolist()
sad_tweets_1     = tweet_emotions_1['content'][tweet_emotions_1['sentiment'] == 'sadness'].values.tolist()
neutral_tweets_1 = tweet_emotions_1['content'][tweet_emotions_1['sentiment'] == 'neutral'].values.tolist()
anger_tweets_1   = tweet_emotions_1['content'][tweet_emotions_1['sentiment'] == 'anger'].values.tolist()

In [6]:
len(happy_tweets_1), len(sad_tweets_1), len(neutral_tweets_1), len(anger_tweets_1)

## Take emotions from 2nd data.
[Emotion Dataset for Emotion Recognition Tasks](https://www.kaggle.com/parulpandey/emotion-dataset)


**sadness (0), joy (1), love (2), anger (3), fear (4).**

In [7]:
tweet_emotions_training_2   = pd.read_csv('../input/emotion-dataset/training.csv')
tweet_emotions_test_2       = pd.read_csv('../input/emotion-dataset/test.csv')
tweet_emotions_validation_2 = pd.read_csv('../input/emotion-dataset/validation.csv')

tweet_emotions_2 = pd.concat([tweet_emotions_training_2,tweet_emotions_test_2,tweet_emotions_validation_2])

In [8]:
labels = {0: 'sadness',
         1: 'joy',
         2: 'love',
         3: 'anger',
         4: 'fear'}

tweet_emotions_2['sentiment']    = tweet_emotions_training_2['label'].map(labels)
tweet_emotions_2.head()

In [9]:
tweet_emotions_2.sentiment.value_counts()

In [10]:
happy_tweets_2   = tweet_emotions_2['text'][tweet_emotions_2['sentiment'] == 'joy'].values.tolist()
sad_tweets_2     = tweet_emotions_2['text'][tweet_emotions_2['sentiment'] == 'sadness'].values.tolist() 
angr_tweets_2    = tweet_emotions_2['text'][tweet_emotions_2['sentiment'] == 'anger'].values.tolist() 
# natural_tweets_2 = tweet_emotions_2['text'][tweet_emotions_2['sentiment'] == 'fear'].values.tolist()                  

In [11]:
len(happy_tweets_2), len(sad_tweets_2), len(angr_tweets_2)

## Take emotions from 3rd data.
[Emotions dataset for NLP](https://www.kaggle.com/praveengovi/emotions-dataset-for-nlp)

<!-- sadness (0), joy (1), love (2), anger (3), fear (4). -->

In [12]:
tweet_emotions_training_3 = pd.read_csv("/kaggle/input/emotions-dataset-for-nlp/train.txt", delimiter=';', header=None, names=['sentence','label'])
tweet_emotions_test_3 = pd.read_csv("/kaggle/input/emotions-dataset-for-nlp/test.txt", delimiter=';', header=None, names=['sentence','label'])
tweet_emotions_validation_3 = pd.read_csv("/kaggle/input/emotions-dataset-for-nlp/val.txt", delimiter=';', header=None, names=['sentence','label'])

tweet_emotions_3 = pd.concat([tweet_emotions_training_3,tweet_emotions_test_3,tweet_emotions_validation_3])
tweet_emotions_3.head()

In [13]:
tweet_emotions_3.label.value_counts()

In [14]:
happy_tweets_3   = tweet_emotions_3['sentence'][tweet_emotions_3['label'] == 'joy'].values.tolist()
sad_tweets_3     = tweet_emotions_3['sentence'][tweet_emotions_3['label'] == 'sadness'].values.tolist() 
angr_tweets_3    = tweet_emotions_3['sentence'][tweet_emotions_3['label'] == 'anger'].values.tolist() 
# angr_tweets_3 = tweet_emotions_3['sentence'][tweet_emotions_3['label'] == 'fear'].values.tolist()                  

In [15]:
len(happy_tweets_3), len(sad_tweets_3), len(angr_tweets_3)

## Compining all in one DataFrame

In [16]:
happy_tweets , sad_tweets , angr_tweets, natural_tweets = [], [], [] , []

happy_tweets   = happy_tweets_1 + happy_tweets_2 + happy_tweets_3
sad_tweets     = sad_tweets_1   + sad_tweets_2   + sad_tweets_3
anger_tweets    = anger_tweets_1  + angr_tweets_2  + angr_tweets_3 
neutral_tweets = neutral_tweets_1


len(happy_tweets), len(sad_tweets), len(anger_tweets), len(neutral_tweets)

In [17]:
# maka list for all emotions and also list of labels for each label with fixied number of tweets for his list
all_labels = [['happy'], ['sad'], ['anger'], ['neutral']]
all_tweets = [happy_tweets, sad_tweets, anger_tweets,neutral_tweets]

tweets = []
labels = []

for idx, label in enumerate(all_tweets):
    curr_size = len(all_tweets[idx])
    curr_label = all_labels[idx]
    tweets += all_tweets[idx]
    labels +=  curr_label * curr_size
    print(curr_label, curr_size)
    

In [18]:
# both have the same lenght
len(tweets), len(labels)

In [19]:
emotion_df = pd.DataFrame({'tweets': tweets,
                          'labels': labels})

# shuffel data 
emotion_df = emotion_df.sample(frac=1).reset_index(drop=True)

emotion_df.head()

In [20]:
# distribute the data by ordering of frequent number of labels

sns.countplot(data = emotion_df, x = 'labels',
             order = emotion_df['labels'].value_counts().index)
plt.title('Distribute the data by ordering of frequent number of labels')
plt.show();

In [21]:
labels=emotion_df['labels'].value_counts().index
values=emotion_df['labels'].value_counts().values

#visualization
plt.figure(figsize=(7,7))
plt.pie(values ,labels = labels ,autopct='%1.1f%%')
plt.title('Percentage of labels for each class')
plt.show()

## Data Wrangling

### Make some text-preprocessing

In [28]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
english_words = set(nltk.corpus.words.words())
    
def remove_emojis(text):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)


# remove non english words
def remove_non_english(text):
    return " ".join(w.lower() for w in nltk.wordpunct_tokenize(text) if w.lower() in english_words or not w.isalpha())


def clean_text(text):
    '''
        Make text lowercase, remove text in square brackets,remove links,remove punctuation
        and remove words containing numbers.
    '''

    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"

    text = re.sub('https?://\S+|www\.\S+', '', text)                # remove urls
    text = re.sub('<.*?>+', '', text)                               # remove tages
    text = re.sub(r'@[^\s]+', ' ', text)                            # Removing @user
    text = re.sub(r'#([^\s]+)', r'\1', text)                        # remove hashtage #word
    text = re.sub('[%s]' % re.escape(punctuations_list), '', text)  # remove punctuation
    text = re.sub('\n', '', text)                                   # remove new line
    text = re.sub(r'\s+', ' ', text)                                # Removing multiple spaces
    text = re.sub(sequencePattern, seqReplacePattern, text)         # Replace 3 or more consecutive letters by 2 letter.
    return text


def preprocess_data(text):
    
    # Clean puntuation, urls, and so on
    text = clean_text(text)

    # Remove emojis
    text = remove_emojis(text)

    # Normalize the text 
    text = remove_non_english(text)
    return text


In [29]:
emotion_df['clean_tweets'] = emotion_df['tweets'].apply(preprocess_data)
emotion_df.head()

**Merge original data and augmented in one emotion data**

## Modeling

In [31]:
emotion_df.shape

In [32]:
# traning 60%, testing 40%
train_df, test_df = train_test_split(emotion_df.loc[:, ['clean_tweets', 'labels']], test_size=0.3, random_state=2022)
len(train_df), len(test_df)

In [33]:
# testing 50%, validation 50% -> from the whole testing data which is 40% of the whole data
test_df, eval_df = train_test_split(test_df, test_size=0.5)
len(test_df), len(eval_df)

In [34]:
len(train_df), len(test_df), len(eval_df)

In [35]:
classes = list(set(emotion_df.labels))
classes

### [Distilbert-base-uncased-emotion Model](https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion?text=I+like+you.+I+love+you)

In [38]:
%%time

MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names = classes)
trn = t.preprocess_train(train_df.clean_tweets.values.tolist(), train_df.labels.values.tolist())
val = t.preprocess_test(eval_df.clean_tweets.values.tolist(), eval_df.labels.values.tolist())
distilbert_model = t.get_classifier()
distilbert_learner = ktrain.get_learner(distilbert_model, train_data=trn, val_data=val, batch_size=6)
distilbert_learner.fit_onecycle(5e-5, 4)
distilbert_learner.validate(class_names=classes) # class_names must be string values

In [39]:
distilbert_predictor = ktrain.get_predictor(distilbert_model, preproc= t)

In [61]:
check = [ "I'm very happy because I get a job",
         "I'm not very happy because I did not get a job yet!!",
         "I'm fine, every thing will be okay.",
         "Internet is painfully slow today",
         ]
distilbert_predictor.predict(check)

**Classification Report Testing data**

In [47]:
X_test = test_df.clean_tweets.values.tolist() 
y_test = test_df.labels.values.tolist()

y_pred = distilbert_predictor.predict(X_test)

print(f'The Results Of Classification Report By DistalBERT for Testing data: \n{classification_report(y_test, y_pred)}')

In [48]:
# save the model
distilbert_predictor.save('emotion_distilbert_predictor')

In [49]:
# load the model
reloaded_predictor = ktrain.load_predictor('emotion_distilbert_predictor')

In [62]:
reloaded_predictor.predict("This is ridiculous")

In [51]:
# import time
# for i in range(10000):
#     print(i)
#     time.sleep(10)
